# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
#Импортируем библиотеки.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
#Считываем данные из csv-файла в датафрейм
try:
    data = pd.read_csv('/datasets/users_behavior.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

In [3]:
#Функция получения первичной информации о датафрейме
def studying_the_data (data):
    print ('------------- Первые 5 строк ------------')
    display(data.head())
    print('')
    print('')
    print ('------------- Типы данных ------------')
    print (data.info())
    print('')
    print('')
    print ('------------- Пропуски ------------')
    count = 0
    for element in data.columns:
        if data[element].isna().sum() > 0: 
            print(element, ' - ', data[element].isna().sum(), 'пропусков')
            count = +1
        if count == 0:
            print(element, 'Пропусков НЕТ')
    print('')
    print('')
    print ('------------- Дубликаты ------------')
    if data.duplicated().sum() > 0:
        print('Дубликатов: ', data.duplicated().sum())
    else:
        print('Дубликатов НЕТ')

studying_the_data (data)

------------- Первые 5 строк ------------


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0




------------- Типы данных ------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


------------- Пропуски ------------
calls Пропусков НЕТ
minutes Пропусков НЕТ
messages Пропусков НЕТ
mb_used Пропусков НЕТ
is_ultra Пропусков НЕТ


------------- Дубликаты ------------
Дубликатов НЕТ


## Разбейте данные на выборки

In [4]:
#Разделите исходные данные на обучающую, валидационную и тестовую выборки.

#Отделим 60% данных для обучающей выборки
data_tmp, data_train = train_test_split(data, test_size=0.60, random_state=12345)

#Разделим оставшуюся часть исходных данных на валидную и тестовую
data_test, data_valid = train_test_split(data_tmp, test_size=0.50, random_state=12345)

#Создадим переменные для признаков и целевого признака
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(1929, 4)
(1929,)
(643, 4)
(643,)
(642, 4)
(642,)


## Исследуйте модели

In [5]:
#Построим модель решающего дерева
best_model = None
best_result = 0
for depth in range(1, 6):
	model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучим модель с заданной глубиной дерева
	model_tree.fit(features_train, target_train) # обучим модель вызовом метода fit()
	predictions = model_tree.predict(features_valid) # получим предсказания модели
	result = accuracy_score(target_valid, predictions) # посчитайте качество модели
	if result > best_result:
		best_model = model_tree
		best_result = result
print("Accuracy наилучшей модели на валидационной выборке:", best_result)        

Accuracy наилучшей модели на валидационной выборке: 0.7900466562986003


In [6]:
#Построим модель случайного леса
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # обучим модель с заданным количеством деревьев
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result = model.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_model = model # сохраним наилучшую модель
        best_result = result #  сохраним наилучшее значение метрики accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692


In [7]:
#Построим модель логистической регрессии
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train) 
print(model.score(features_train, target_train))

0.7511664074650077


Самый лучший результать получили при модели решающего дерева, доля правильных ответов 79%.

## Проверьте модель на тестовой выборке

In [8]:
predictions = model_tree.predict(features_test)
result = accuracy_score(target_test, predictions)
print(result)

0.7710280373831776


77% - немного хуже, чем на валидной выборке.

## (бонус) Проверьте модели на адекватность

In [9]:
model = DummyClassifier().fit(features_train, target_train)
print(model.score(features_train, target_train))

0.7081389320891653


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75


# Вывод

Мы построили 3 модели: решающее дерево, случайный лес, логистическая регрессия. Получили максимально высокое качество предсказания перебирая в алгоритме обучения разные значения гиперпараметров. Самый лучший результать получили при модели решающего дерева, доля правильных ответов 79%. Проверили модель на тестовой выборке и получили хороший результат - 77%.

Тем самым мы построили модель для задачи классификации, которая выберет подходящий тариф.